In [40]:
import matplotlib.pyplot as plt
import numpy as np
import os
import tensorflow as tf
import tensorflow_hub as hub
import cv2
import shutil
import random

In [59]:
# Set some paths
# Download images to a folder in this directory and set PATH variable to its path
PATH = "/Users/jeremystubbs/Desktop/Python/Machine_Learning_NNs/Mobilenet/mobilenet_custom/archive"
train_dir = os.path.join(PATH, 'train')
validation_dir = os.path.join(PATH, 'valid')
images_dir = os.path.join(PATH, "images.csv")



In [ ]:
if os.path.isdir(train_dir) is False:
    os.mkdir(train_dir)
if os.path.isdir(validation_dir) is False:
    os.mkdir(validation_dir)

# Create list of all unique labels from the csv file
with open(images_dir) as f:
    all_image_paths = [row.split(",")[0] for row in f]
with open(images_dir) as f:
    all_image_labels = [row.split(",")[2] for row in f] 

unique_labels = set(all_image_labels)

# Make subfolders for each class
for item in unique_labels:
  if os.path.isdir('train/'+str(item)) is False:
    os.mkdir('train/'+str(item))
  if os.path.isdir('valid/'+str(item)) is False:
    os.mkdir('valid/'+str(item))

In [ ]:
# Move all pictures to images_folder subfolders - change second part to your folder name
IMAGES_PATH = os.path.join(PATH, 'images_original')
for item in unique_labels:
    # print(item)
    temp_arr = []
    if item == "label":
        continue
    for i in range(len(all_image_labels)):
        if all_image_labels[i]==item:
            temp_arr.append(all_image_paths[i])

    k = int(len(temp_arr) * 0.8)
    train_temp_arr = random.sample(temp_arr, k)  
    valid_temp_arr = list(set(temp_arr).difference(train_temp_arr))
    # print(len(train_temp_arr), len(valid_temp_arr))
    # print(IMAGES_PATH+train_temp_arr[0]+".jpg")
    # print(valid_temp_arr[0])

    for val in train_temp_arr:
        try:
            shutil.copy(IMAGES_PATH +"/"+ val+".jpg", train_dir+"/"+item)
        except:
            x=2

    for each in valid_temp_arr:
        try:
            shutil.copy(IMAGES_PATH +"/"+ each+".jpg", validation_dir+"/"+item)
        except:
            x=2


In [60]:
# Make training and validation Datasets
BATCH_SIZE = 10
IMG_SIZE = (224, 224)

# train_dataset = tf.keras.utils.image_dataset_from_directory(train_dir,
#                                                             labels = "inferred",
#                                                             shuffle=True,
#                                                             batch_size=BATCH_SIZE,
#                                                             image_size=IMG_SIZE)

# validation_dataset = tf.keras.utils.image_dataset_from_directory(validation_dir,
#                                                                 labels = "inferred",
#                                                                 shuffle=True,
#                                                                 batch_size=BATCH_SIZE,
#                                                                 image_size=IMG_SIZE)
# class_names = train_dataset.class_names
train_dataset = tf.keras.preprocessing.image.ImageDataGenerator().flow_from_directory(directory=train_dir, target_size=IMG_SIZE, batch_size=BATCH_SIZE)
validation_dataset = tf.keras.preprocessing.image.ImageDataGenerator().flow_from_directory(directory=validation_dir,target_size=IMG_SIZE, batch_size=BATCH_SIZE)




Found 14034 images belonging to 6 classes.
Found 0 images belonging to 0 classes.


In [ ]:
# Plot some images
plt.figure(figsize=(10, 10))
for images, labels in train_dataset.take(1):
  for i in range(9):
    ax = plt.subplot(3, 3, i + 1)
    plt.imshow(images[i].numpy().astype("uint8"))
    plt.title(class_names[labels[i]])
    plt.axis("off")

In [43]:
# Define preprocessing function and create train and validation iterators

preprocess_input = tf.keras.applications.mobilenet_v2.preprocess_input

# This does the same thing
# rescale = tf.keras.layers.Rescaling(1./127.5, offset=-1)

# Could include preprocessing here
# train_dataset = tf.keras.preprocessing.image.ImageDataGenerator(preprocessing_function=preprocess_input).flow_from_directory(directory=train_dir, target_size=(224,224), batch_size=10)
# validation_dataset = tf.keras.preprocessing.image.ImageDataGenerator(preprocessing_function=preprocess_input).flow_from_directory(directory=validation_dir,target_size=(224,224), batch_size=10)


image_batch, label_batch = next(iter(train_dataset))


In [44]:
# Create the base model from the pre-trained model MobileNet V2
IMG_SHAPE = IMG_SIZE + (3,)

base_model = tf.keras.applications.MobileNetV2(input_shape=IMG_SHAPE,
                                               include_top=False,
                                               weights='imagenet')

base_model.trainable = False



In [45]:
##This cell creates the model
# Add input, preprocessing, global average and prediction layer to the base model
# feature_batch, and prediction_batch are optional 
# feature_batch = base_model(image_batch)
global_average_layer = tf.keras.layers.GlobalAveragePooling2D()
# feature_batch_average = global_average_layer(feature_batch)
# prediction_layer = tf.keras.layers.Dense(len(class_names),
                        #   kernel_regularizer=tf.keras.regularizers.l2(0.0001))
# prediction_batch = prediction_layer(feature_batch_average)
prediction_layer = tf.keras.layers.Dense(units=13, activation='softmax')
identity_layer = tf.keras.layers.Layer()

inputs = tf.keras.Input(shape=(224, 224, 3))
x = preprocess_input(inputs)
x = base_model(x, training=False)
x = identity_layer(x)
x = global_average_layer(x)
x = tf.keras.layers.Dropout(0.2)(x)
outputs = prediction_layer(x)
model = tf.keras.Model(inputs, outputs)



In [46]:
model.summary()

Model: "model_7"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_13 (InputLayer)       [(None, 224, 224, 3)]     0         
                                                                 
 tf.math.truediv_9 (TFOpLamb  (None, 224, 224, 3)      0         
 da)                                                             
                                                                 
 tf.math.subtract_9 (TFOpLam  (None, 224, 224, 3)      0         
 bda)                                                            
                                                                 
 mobilenetv2_1.00_224 (Funct  (None, 7, 7, 1280)       2257984   
 ional)                                                          
                                                                 
 layer_4 (Layer)             (None, 7, 7, 1280)        0         
                                                           

In [47]:
##Compile the model
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])

# Here is an alternative setup
# model.compile(
#   optimizer=tf.keras.optimizers.SGD(learning_rate=0.005, momentum=0.9), 
#   loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True, label_smoothing=0.1),
#   metrics=['accuracy'])

# model.summary()



In [48]:
#Train the model
initial_epochs = 10

model.fit(train_dataset,
                    epochs=initial_epochs, 
                    validation_data=validation_dataset, )



Epoch 1/10
372/372 [==============================] - 274s 734ms/step - loss: 1.9421 - accuracy: 0.4047 - val_loss: 1.4428 - val_accuracy: 0.5636
Epoch 2/10
372/372 [==============================] - 273s 733ms/step - loss: 1.3272 - accuracy: 0.5859 - val_loss: 1.1186 - val_accuracy: 0.6556
Epoch 3/10
372/372 [==============================] - 273s 734ms/step - loss: 1.0693 - accuracy: 0.6621 - val_loss: 0.9547 - val_accuracy: 0.7091
Epoch 4/10
372/372 [==============================] - 273s 735ms/step - loss: 0.9316 - accuracy: 0.7073 - val_loss: 0.8644 - val_accuracy: 0.7198
Epoch 5/10
372/372 [==============================] - 274s 736ms/step - loss: 0.8359 - accuracy: 0.7359 - val_loss: 0.7901 - val_accuracy: 0.7529
Epoch 6/10
372/372 [==============================] - 273s 736ms/step - loss: 0.7646 - accuracy: 0.7536 - val_loss: 0.7448 - val_accuracy: 0.7636
Epoch 7/10
372/372 [==============================] - 274s 736ms/step - loss: 0.7072 - accuracy: 0.7795 - val_loss: 0.7104 -

In [52]:
# Optional fine-tuning I read on tensorflow's official website
# Let's take a look to see how many layers are in the base model
print("Number of layers in the base model: ", len(base_model.layers))

# # Fine-tune from this layer onwards
fine_tune_at = 100

base_model.trainable = True
# # Freeze all the layers before the `fine_tune_at` layer
for layer in base_model.layers[:fine_tune_at]:
  layer.trainable = False

Number of layers in the base model:  154


In [54]:
model.summary()

Model: "model_7"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_13 (InputLayer)       [(None, 224, 224, 3)]     0         
                                                                 
 tf.math.truediv_9 (TFOpLamb  (None, 224, 224, 3)      0         
 da)                                                             
                                                                 
 tf.math.subtract_9 (TFOpLam  (None, 224, 224, 3)      0         
 bda)                                                            
                                                                 
 mobilenetv2_1.00_224 (Funct  (None, 7, 7, 1280)       2257984   
 ional)                                                          
                                                                 
 layer_4 (Layer)             (None, 7, 7, 1280)        0         
                                                           

In [55]:
##Recompile the model with a lower learning rate
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.00001), loss='categorical_crossentropy', metrics=['accuracy'])


In [57]:
#Train the model
initial_epochs = 10

model.fit(train_dataset,
                    epochs=initial_epochs, 
                    validation_data=validation_dataset, )

Epoch 1/10
  5/372 [..............................] - ETA: 3:25 - loss: 0.0705 - accuracy: 0.9800

KeyboardInterrupt: 

In [58]:
##Save model
os.chdir("/Users/jeremystubbs/Desktop/Python/Machine_Learning_NNs/Mobilenet/mobilenet_custom/")
!mkdir -p saved_model
model.save('saved_model/my_model')

INFO:tensorflow:Assets written to: saved_model/my_model/assets


/opt/homebrew/Caskroom/miniforge/base/envs/tf/lib/python3.10/site-packages/keras/engine/functional.py:1410: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  layer_config = serialize_layer_fn(layer)
/opt/homebrew/Caskroom/miniforge/base/envs/tf/lib/python3.10/site-packages/keras/saving/saved_model/layer_serialization.py:112: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  return generic_utils.serialize_keras_object(obj)


In [65]:
## Predict images in test set with model 
label_map = ['Blazer','Dress','Hat','Hoodie', 'Longsleeve','Pants', 'Polo','Shirt','Shoes','Shorts', 'Skirt','T-Shirt','Undershirt']
threshold = 0.5
# img = cv2.imread("test_images/blazer.jpeg")
img = cv2.imread("test_images/dress.jpg")
resized = cv2.resize(img, (224, 224))
reshaped = resized.reshape(1, 224, 224, 3)

predictions = model.predict(reshaped, verbose=0)
for val in predictions[0]:
    if val > threshold:
        index_of = np.where(predictions[0]==val)
        print(label_map[index_of[0][0]])
print(predictions)


Dress
[[1.0351728e-05 9.8145181e-01 1.2068073e-03 3.7743028e-03 8.6038090e-05
  2.1551554e-05 5.7943520e-04 3.3679139e-03 9.2482949e-03 2.8239296e-05
  1.9095450e-04 2.0961739e-05 1.3441330e-05]]
